## Transfer Learning from InceptionV3

The idea of this step is to leverage existing state of the art models such as Ince

Borrowed some details from this example [https://www.kaggle.com/kmader/transfer-learning-with-inceptionv3](https://www.kaggle.com/kmader/transfer-learning-with-inceptionv3)

# Create image generators

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from itertools import chain

In [4]:
def generate_generator_multiple(generator,directories, batch_size, img_height,img_width):
    generators =[]
    for directory in directories:
        gen = generator.flow_from_directory(directory,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
        generators.append(gen)

    for gen in generators:
        for data, labels in gen:
            yield data, labels

            


# Doing K-Folds

In [5]:
batch_size = 32
img_height=128
img_width = 150
approx_fold_size = 800

In [6]:
urban_sounds_folder = "/media/romulo/6237-3231/urban_sound_challenge/spectrograms"

In [7]:
fold_directories = []
for i in range(1,11):
    directory = urban_sounds_folder+"/fold"+str(i)
    fold_directories.append(directory)

In [8]:
fold_directories

['/media/romulo/6237-3231/urban_sound_challenge/spectrograms/fold1',
 '/media/romulo/6237-3231/urban_sound_challenge/spectrograms/fold2',
 '/media/romulo/6237-3231/urban_sound_challenge/spectrograms/fold3',
 '/media/romulo/6237-3231/urban_sound_challenge/spectrograms/fold4',
 '/media/romulo/6237-3231/urban_sound_challenge/spectrograms/fold5',
 '/media/romulo/6237-3231/urban_sound_challenge/spectrograms/fold6',
 '/media/romulo/6237-3231/urban_sound_challenge/spectrograms/fold7',
 '/media/romulo/6237-3231/urban_sound_challenge/spectrograms/fold8',
 '/media/romulo/6237-3231/urban_sound_challenge/spectrograms/fold9',
 '/media/romulo/6237-3231/urban_sound_challenge/spectrograms/fold10']

# MODEL

In [9]:
datagen = ImageDataGenerator(rescale=1./255)

In [17]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications
from keras import metrics
input_shape = (img_height, img_width,3)
nclass = 10

base_model = applications.InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(img_height, img_width,3))
base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dropout(0.5))
add_model.add(Dense(nclass, 
                    activation='softmax'))

model = add_model
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.SGD(lr=1e-4, 
                                       momentum=0.9),
              metrics=['accuracy'])
#needed to reset weights!
model.save_weights('raw_model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 2, 3, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                20490     
Total params: 21,823,274
Trainable params: 20,490
Non-trainable params: 21,802,784
_________________________________________________________________


# Folding

In [ ]:
num_folds = 5 #1 to 10
fold = 0
for directory in fold_directories:
    fold +=1
    #RESET WEIGHTS!!
    model.load_weights('raw_model.h5')
    #
    train_directories = list(set(fold_directories) - set([directory]))
    test_directories = [directory]
    print("Running fold {}, holding data from {} and training on the remaining {}" \
          .format(fold,directory,len(train_directories)))
    train_generator = generate_generator_multiple(generator=datagen,
                                           directories = train_directories,
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_width)
    test_generator = generate_generator_multiple(generator=datagen,
                                       directories = test_directories,
                                       batch_size=batch_size,
                                       img_height=img_height,
                                       img_width=img_width)
    history = model.fit_generator(train_generator,
                              steps_per_epoch=approx_fold_size/batch_size,
                              epochs=20,
                              validation_data = test_generator,
                              validation_steps=approx_fold_size/batch_size,
                              shuffle=True, 
                              verbose=True)
    model.save_weights('trained_model_fold_{}.h5'.format(fold))
    if fold == num_folds:
        break

Running fold 1, holding data from /media/romulo/6237-3231/urban_sound_challenge/spectrograms/fold1 and training on the remaining 9
Epoch 1/20
Found 873 images belonging to 10 classes.
Found 823 images belonging to 10 classes.
Found 837 images belonging to 10 classes.
Found 990 images belonging to 10 classes.
Found 838 images belonging to 10 classes.
Found 936 images belonging to 10 classes.
Found 888 images belonging to 10 classes.
Found 816 images belonging to 10 classes.
Found 925 images belonging to 10 classes.
Found 806 images belonging to 10 classes.
25/25 [==============================] - 86s 3s/step - loss: 2.7596 - acc: 0.0888 - val_loss: 2.8195 - val_acc: 0.1187
Epoch 2/20
25/25 [==============================] - 78s 3s/step - loss: 2.6836 - acc: 0.1013 - val_loss: 2.9619 - val_acc: 0.1094
Epoch 3/20
25/25 [==============================] - 80s 3s/step - loss: 2.6853 - acc: 0.0997 - val_loss: 2.8956 - val_acc: 0.1055
Epoch 4/20
25/25 [==============================] - 77s 3s/